# "Uploading" PDFs to Claude Via the API

https://github.com/anthropics/anthropic-cookbook/blob/main/misc/pdf_upload_summarization.ipynb


One really nice feature of Claude.ai is the ability to upload PDFs. Let's mock up that feature in a notebook, and then test it out by summarizing a long PDF.

We'll start by installing the Anthropic client and create an instance of it we will use throughout the notebook.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key using os.getenv()
claude_api_key = os.getenv("ANTHROPIC_API_KEY")

In [6]:
from anthropic import Anthropic
# While PDF support is in beta, you must pass in the correct beta header
client = Anthropic(api_key=claude_api_key, default_headers={
    "anthropic-beta": "pdfs-2024-09-25"
  }
)
# For now, only claude-3-5-sonnet-20241022 supports PDFs
MODEL_NAME = "claude-3-5-sonnet-20241022"

We already have a PDF available in the ../multimodal/documents directory. We'll convert the PDF file into base64 encoded bytes. This is the format required for the PDF document block in the Anthropic API. Note that this type of extraction works for both text and visual elements (like charts and graphs).

In [3]:
import base64

# Start by reading in the PDF and encoding it as base64
file_name = "../documents/constitutional-ai-paper.pdf"
with open(file_name, "rb") as pdf_file:
  binary_data = pdf_file.read()
  base64_encoded_data = base64.standard_b64encode(binary_data)
  base64_string = base64_encoded_data.decode("utf-8")

With the paper downloaded and in memory, we can ask Claude to perform various fun tasks with it. We'll pass the document ot the model alongside a simple question.

In [4]:
prompt = """
Please do the following:
1. Summarize the abstract at a kindergarten reading level. (In <kindergarten_abstract> tags.)
2. Write the Methods section as a recipe from the Moosewood Cookbook. (In <moosewood_methods> tags.)
3. Compose a short poem epistolizing the results in the style of Homer. (In <homer_results> tags.)
"""
messages = [
    {
        "role": 'user',
        "content": [
            {"type": "document", "source": {"type": "base64", "media_type": "application/pdf", "data": base64_string}},
            {"type": "text", "text": prompt}
        ]
    }
]

In [7]:
def get_completion(client, messages):
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=messages
    ).content[0].text
    
completion = get_completion(client, messages)
print(completion)

Let me tackle each of these creative reinterpretations:

<kindergarten_abstract>
Scientists are trying to teach computers to be nice and helpful! They want computers to help watch over other computers, like a good big sister or brother. Instead of having humans tell the computers exactly what to do, they give the computers some simple rules to follow. The computers learn to be nice by practicing and fixing their mistakes. In the end, they created a computer helper that is both nice and honest!
</kindergarten_abstract>

<moosewood_methods>
Constitutional AI Stew
(A nourishing recipe for training helpful AI systems)

Ingredients:
- 1 helpful AI model
- A bundle of constitutional principles
- Several cups of training data
- A dash of supervised learning
- A sprinkle of reinforcement learning

Method:
1. Begin by gently simmering your helpful AI model in a bath of training examples.
2. In a separate bowl, whisk together your constitutional principles until well-blended.
3. Gradually fold i